In [1]:
import tensorflow as tf
import numpy as np

C:\Users\haho6\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [311]:
from keras.preprocessing.image import load_img, img_to_array
images = []
for i in range(70):
    image = load_img('./DATA/'+str(i)+'.jpeg',target_size=(64,64))
    image = img_to_array(image)
    image.shape
    image = image / 256
    images.append(image)
    images.append(image)
    images.append(image)

In [312]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size = [batch_size, n_noise])

In [313]:
def windowUp(arr):
    import cv2
    frame = cv2.cvtColor(np.array(arr), cv2.COLOR_BGR2RGB)
    cv2.imshow("test", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [314]:
X = tf.placeholder(tf.float32, [None, 64,64,3])
Y = tf.placeholder(tf.float32, [None, 1])
Z = tf.placeholder(tf.float32, [None, 100])

In [315]:
def generate(Z):
    w1 = tf.Variable(tf.random_normal([100, 16384]))
    h1 = tf.matmul(Z,w1)
    re = tf.reshape(h1,[-1,4,4,1024])
    re = tf.layers.batch_normalization(re)

    conv_h1 = tf.layers.conv2d_transpose(re ,512, 5, 2, padding='same')
    
    conv_h1 = tf.nn.relu(conv_h1)

    conv_h2 = tf.layers.conv2d_transpose(conv_h1 ,256, 4, 2, padding='same')
    conv_h2 = tf.nn.relu(conv_h2)

    conv_h3 = tf.layers.conv2d_transpose(conv_h2 ,128, 4, 2, padding='same')
    conv_h3 = tf.nn.relu(conv_h3)

    conv_h5 = tf.layers.conv2d_transpose(conv_h3 ,3, 4, 2, padding='same')
    return conv_h5

In [316]:
generate(Z)

<tf.Tensor 'conv2d_transpose_128/BiasAdd:0' shape=(?, 64, 64, 3) dtype=float32>

In [317]:
def discriminaster(inputs):
    W1 = tf.Variable(tf.random_normal([3,3,3,32],stddev=0.02))
    H1 = tf.nn.conv2d(inputs,W1, strides=[1,2,2,1], padding="SAME")
    H1 = tf.layers.batch_normalization(H1)
    H1 = tf.nn.relu(H1)

    W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.02))
    H2 = tf.nn.conv2d(H1,W2, strides=[1,2,2,1], padding="SAME")
    H2 = tf.nn.relu(H2)

    W3 = tf.Variable(tf.random_normal([3,3,64,256],stddev=0.02))
    H3 = tf.nn.conv2d(H2,W3, strides=[1,2,2,1], padding="SAME")
    H3 = tf.nn.relu(H3)
    
    W4 = tf.Variable(tf.random_normal([3,3,256,1024],stddev=0.02))
    H4 = tf.nn.conv2d(H3,W4, strides=[1,2,2,1], padding="SAME")
    H4 = tf.nn.relu(H4)
    
    out = tf.reshape(H4,[-1,16384])
    out = tf.layers.dense(out,256)
    out = tf.layers.dense(out,1)
    return out

In [318]:
discriminaster(X)

<tf.Tensor 'dense_44/BiasAdd:0' shape=(?, 1) dtype=float32>

In [319]:
G = generate(Z)
D_real = discriminaster(X)
D_gene = discriminaster(G)

In [320]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real,labels=tf.ones_like(D_real)))

loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene,labels=tf.zeros_like(D_gene)))

In [321]:
loss_D = loss_D_gene + loss_D_real
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [322]:
train_D = tf.train.AdamOptimizer(0.001).minimize(loss_D)
train_G = tf.train.AdamOptimizer(0.001).minimize(loss_G)

In [323]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(100):
    noise = get_noise(1,100)
    _, varsD=sess.run([train_D, loss_D], 
                  feed_dict={
                      X:np.array(images),
                      Z: noise})
    _, varsG=sess.run([train_G, loss_G], 
                  feed_dict={
                      X:np.array(images),
                      Z: noise})
    print(str(i)+"   D : "+str(varsD)+"\t G : "+str(varsG))


0   D : 0.07073806	 G : 7.2089667
1   D : 0.50758207	 G : 4.7999544
2   D : 0.4613163	 G : 4.5590897
3   D : 0.5115844	 G : 5.399264
4   D : 0.3152466	 G : 6.12383
5   D : 0.10904441	 G : 7.3741455
6   D : 0.13261847	 G : 7.2677674
7   D : 0.08520225	 G : 7.6240573
8   D : 0.26991147	 G : 6.215007
9   D : 0.18053775	 G : 6.558314
10   D : 0.51203775	 G : 4.836068
11   D : 0.13179043	 G : 6.8006134
12   D : 0.21708366	 G : 6.186818
13   D : 0.23863378	 G : 6.165909
14   D : 0.49845076	 G : 4.97559
15   D : 0.6637564	 G : 5.1771126
16   D : 0.071461864	 G : 8.373516
17   D : 0.14756085	 G : 6.819016
18   D : 0.3144809	 G : 6.4568844
19   D : 0.28568918	 G : 6.4791083
20   D : 0.15415771	 G : 6.4629035
21   D : 0.1226498	 G : 6.7484255
22   D : 0.29426476	 G : 6.4842587
23   D : 0.10502135	 G : 7.6078663
24   D : 0.1682503	 G : 6.730981
25   D : 0.0610506	 G : 7.910382
26   D : 0.10695939	 G : 6.8418837
27   D : 0.7690029	 G : 4.595644
28   D : 0.2920626	 G : 5.738755
29   D : 0.69480026	

In [342]:
noise = get_noise(1,100)
i = sess.run(G, feed_dict={Z:noise})
windowUp(i[0])

In [105]:
i.shape

(1, 128, 128, 3)

In [207]:
noise = get_noise(1,100)